In [70]:
from DATA_INDICATORS import *
import datetime
import time

In [71]:
import ccxt
class AutoTrade:
    def __init__(self,api_key,sec_key,target_symbol = "BTC/USDT",leverage = 5):
        self.api_key = api_key
        self.sec_key = sec_key
        self.target_symbol = target_symbol
        self.leverage = leverage
        try:
            self.binance_connect = ccxt.binance(config={
            'apiKey': api_key, 
            'secret': sec_key,
            'enableRateLimit': True,
            'options': {
                'defaultType': 'future'
                }
            })
            print("binance connect complete")
        except:
            print("binance connect error")
    def download_data(self,frame):
        try:
            binance = ccxt.binance()
            btc_ohlcv = binance.fetch_ohlcv(self.target_symbol,limit = 100,timeframe=frame)
            df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
            df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
            df.set_index('datetime', inplace=True)
            return df
        except:
            print("data get error")
    # 잔고 조회
    def CHECK_YOUR_BALANCE(self):
        print("총 증거금 :",self.binance_connect.fetch_balance()['info']['totalInitialMargin'],"USDT") # 총 증거금
        print("총 지갑 잔액 :",self.binance_connect.fetch_balance()['info']['totalInitialMargin'],"USDT") # 총 지갑 잔액 
        print("총 사용 가능 잔액 :",self.binance_connect.fetch_balance()['info']['availableBalance'],"USDT") # 사용 가능 잔액 
        print("출금 가능 금액 :",self.binance_connect.fetch_balance()['info']['maxWithdrawAmount'],"USDT") # 출금 가능 금액

    # 타겟 심볼 레버리지 변경 
    def change_leverage(self,leverage = 5):
        symbol = self.target_symbol
        try:
            market = self.binance_connect.market(str(symbol))
            self.leverage = leverage
            leverage = self.leverage
            try:
                resp = self.binance_connect.fapiPrivate_post_leverage({
                    'symbol': market['id'],
                    'leverage': leverage
                })
                print("change",self.leverage,"laverage complete")
            except:
                print("chane leverage error")
        except:
            print("No Symbol in market")
    # 타겟 심볼 평단가 조회
    def entryprice(self):
        balance = self.binance_connect.fetch_balance()
        positions = balance['info']['positions']

        for position in positions:
            if position["symbol"] == self.target_symbol:
                return float(position['entryPrice'])

    # rsi 기본 매매
    def rsi_auto_trade(self,period = 14,frame = "1h",buy_num = 30,sell_num = 70,buy_amount = 0.01,st_num = 5):
        buy_flag = True
        entryPrice = 0
        st_num = st_num/100
        while(True):
            print("------------------------------------------------------------------------------------")
            current_time = datetime.now()
            data = self.download_data(frame = frame)
            data = add_rsi(data,data_name = 'close',period = period)
            rsi_indicator_name = 'rsi'+'_'+str(period)
            print("현재 시간 :",current_time)
            print("현재 가격 :",data.iloc[-1]['close'])
            print("현재 rsi :",data.iloc[-1][rsi_indicator_name])
            if buy_flag:
                if data.iloc[-1][rsi_indicator_name] < buy_num:
                    entryPrice = self.entryprice()
                    buy_flag = False

                    price = self.entryprice()
                    print('매수시점:',current_time,'진입가격:',price)

                    order = self.binance_connect.create_market_buy_order(symbol=self.target_symbol,amount=buy_amount)
            else:
                if data.iloc[-1][rsi_indicator_name] > sell_num:
                    entryPrice = 0
                    buy_flag = True

                    price = self.entryprice()
                    print('매도시점:',current_time,'매도가격:',price)

                    order = self.binance_connect.create_market_sell_order(
                    symbol=self.target_symbol,
                    amount=buy_amount)
                # -st_num% 손실 시 매도
            if entryPrice * (1-st_num) >= data.iloc[-1]['close'] and buy_flag:
                order = self.binance.create_market_sell_order(
                symbol=self.target_symbol,
                amount=buy_amount)
                
                price = self.entryprice()
                print('매도시점:',current_time,'매도가격:',price)
            print("------------------------------------------------------------------------------------")
            time.sleep(15)

In [72]:
a = AutoTrade('598lqCWX727NXfflVt1qejRSZXphSuWVlcdMm1gdtAVD3lytdDhjpbV3eyxcr0z1','XxDx9DuVDiYngMV71jWHHCFYRhhph4bdZdGCSH4JVb2s2jCJtHW6gbpPwZWjfYvf')

binance connect complete


In [73]:
a.rsi_auto_trade()

------------------------------------------------------------------------------------
현재 가격 : 20054.54
현재 rsi : 49.28520502573544
------------------------------------------------------------------------------------
------------------------------------------------------------------------------------
현재 가격 : 20048.0
현재 rsi : 48.78047010922315
------------------------------------------------------------------------------------
------------------------------------------------------------------------------------
현재 가격 : 20048.59
현재 rsi : 48.826416221662136
------------------------------------------------------------------------------------
------------------------------------------------------------------------------------
현재 가격 : 20046.86
현재 rsi : 48.69145874736143
------------------------------------------------------------------------------------
